In [57]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt

In [58]:
file_path = 'longgrbs_ready_for_imputation.xlsx'  
df = pd.read_excel(file_path)

like three datapoints were problematic after the knn imputation when i manually inspected the excel sheet. there was one outlier significantly changing the values.


In [59]:
#here we prepare the data for knn imputation
#we found an outlier
flux_col = 'XRT Early Flux (0.3-10 keV) [10^-11 erg/cm^2/s]'
median_flux = df[flux_col].median()

median_magnitude = df['uvot_mag'].max()  
#super weird outlier at this index.
df.at[392, flux_col] = median_flux

#there were two outliers for some reason registering at 198.9 for the uvot magnitude value.
df.at[146, 'uvot_mag'] = 19.89
df.at[193, 'uvot_mag'] = 19.89

first imputation method is to use the median of the column as the imputed value for missing data

In [60]:
df_imputed_median = df.fillna(df.median(numeric_only=True))

second imputation method is to use knn imputation.

In [61]:
#set up and scale data for imputation:
from sklearn.preprocessing import StandardScaler
from sklearn.impute import KNNImputer

impute_cols = df.drop(columns=['GRB', 'Redshift']).columns

scaler = StandardScaler()
df_scaled = pd.DataFrame(scaler.fit_transform(df[impute_cols]), columns=impute_cols)

In [ ]:
imputer = KNNImputer(n_neighbors=7, weights='distance')
df_imputed_scaled = pd.DataFrame(imputer.fit_transform(df_scaled), columns=impute_cols)

df_imputed = pd.DataFrame(scaler.inverse_transform(df_imputed_scaled), columns=impute_cols)

df_imputed_knn = pd.concat([df[['GRB', 'Redshift']].reset_index(drop=True), df_imputed.reset_index(drop=True)], axis=1)
print('knn imputation complete')

knn imputation complete


i will try MICE imputation next and put it below

In [63]:
#required packages are below:
#https://scikit-learn.org/stable/modules/impute.html
#documentation for mice imputation is above.
from sklearn.experimental import enable_iterative_imputer
from sklearn.impute import IterativeImputer
from sklearn.linear_model import BayesianRidge
from sklearn.ensemble import RandomForestRegressor, GradientBoostingRegressor

In [64]:
iteration_number = 30

In [65]:
imputer = IterativeImputer(max_iter=iteration_number, estimator=BayesianRidge())
df_imputed_scaled = pd.DataFrame(imputer.fit_transform(df_scaled), columns=impute_cols)
df_imputed = pd.DataFrame(scaler.inverse_transform(df_imputed_scaled), columns=impute_cols)
df_imputed_mice = pd.concat([df[['GRB', 'Redshift']].reset_index(drop=True), df_imputed.reset_index(drop=True)], axis=1)
print('mice imputation done. with iteration number', iteration_number)

mice imputation done. with iteration number 30


/opt/anaconda3/lib/python3.11/site-packages/sklearn/impute/_iterative.py:785: ConvergenceWarning: [IterativeImputer] Early stopping criterion not reached.
  warnings.warn(


In [66]:
'''imputer_rf = IterativeImputer(
    estimator=RandomForestRegressor(n_estimators=100),
    max_iter=iteration_number,
)

df_imputed_scaled = pd.DataFrame(imputer_rf.fit_transform(df_scaled), columns=impute_cols)
df_imputed = pd.DataFrame(scaler.inverse_transform(df_imputed_scaled), columns=impute_cols)
df_imputed_final = pd.concat([df[['GRB', 'Redshift']].reset_index(drop=True), df_imputed.reset_index(drop=True)], axis=1)
df_imputed_final.to_excel('rf_mice_imputation.xlsx', index=False)

print('mice with random forest imputer done. with iteration number', iteration_number)'''

"imputer_rf = IterativeImputer(\n    estimator=RandomForestRegressor(n_estimators=100),\n    max_iter=iteration_number,\n)\n\ndf_imputed_scaled = pd.DataFrame(imputer_rf.fit_transform(df_scaled), columns=impute_cols)\ndf_imputed = pd.DataFrame(scaler.inverse_transform(df_imputed_scaled), columns=impute_cols)\ndf_imputed_final = pd.concat([df[['GRB', 'Redshift']].reset_index(drop=True), df_imputed.reset_index(drop=True)], axis=1)\ndf_imputed_final.to_excel('rf_mice_imputation.xlsx', index=False)\n\nprint('mice with random forest imputer done. with iteration number', iteration_number)"

In [67]:
'''imputer_gb = IterativeImputer(
    estimator=GradientBoostingRegressor(n_estimators=100, learning_rate=0.1),
    max_iter=iteration_number,
)

df_imputed_scaled = pd.DataFrame(imputer_gb.fit_transform(df_scaled), columns=impute_cols)
df_imputed = pd.DataFrame(scaler.inverse_transform(df_imputed_scaled), columns=impute_cols)
df_imputed_final = pd.concat([df[['GRB', 'Redshift']].reset_index(drop=True), df_imputed.reset_index(drop=True)], axis=1)

df_imputed_final.to_excel('gb_mice_imputation.xlsx', index=False)

print('mice with gradient boosting imputer done. with iteration number', iteration_number)'''

"imputer_gb = IterativeImputer(\n    estimator=GradientBoostingRegressor(n_estimators=100, learning_rate=0.1),\n    max_iter=iteration_number,\n)\n\ndf_imputed_scaled = pd.DataFrame(imputer_gb.fit_transform(df_scaled), columns=impute_cols)\ndf_imputed = pd.DataFrame(scaler.inverse_transform(df_imputed_scaled), columns=impute_cols)\ndf_imputed_final = pd.concat([df[['GRB', 'Redshift']].reset_index(drop=True), df_imputed.reset_index(drop=True)], axis=1)\n\ndf_imputed_final.to_excel('gb_mice_imputation.xlsx', index=False)\n\nprint('mice with gradient boosting imputer done. with iteration number', iteration_number)"

In [ ]:
def engineer_grb_features(df):
    epsilon = 1e-8
    df = df.copy()
    df['flux_ratio_11h_early'] = df['XRT 11 Hour Flux (0.3-10 keV) [10^-11 erg/cm^2/s]'] / (df['XRT Early Flux (0.3-10 keV) [10^-11 erg/cm^2/s]'] + epsilon)
    df['flux_ratio_24h_11h'] = df['XRT 24 Hour Flux (0.3-10 keV) [10^-11 erg/cm^2/s]'] / (df['XRT 11 Hour Flux (0.3-10 keV) [10^-11 erg/cm^2/s]'] + epsilon)
    df['log_time_ratio_uvot_xrt'] = (
        np.log10(np.abs(df['UVOT Time to First Observation [sec]'] + epsilon)) -
        np.log10(np.abs(df['XRT Time to First Observation [sec]'] + epsilon))
    )
    det_columns = ['B_det', 'U_det', 'UVW1_det', 'UVM2_det', 'UVW2_det', 'White_det', 'uvot_detection_flag']
    df['uvot_det_sum'] = df[det_columns].sum(axis=1)
    df['XRT Time to First Observation [sec]'] = np.log10(np.abs(df['XRT Time to First Observation [sec]'] + epsilon))
    df['UVOT Time to First Observation [sec]'] = np.log10(np.abs(df['UVOT Time to First Observation [sec]'] + epsilon))
    df['BAT T90 [sec]'] = np.log10(np.abs(df['BAT T90 [sec]'] + epsilon))
    df['XRT Column Density (NH) [10^21 cm^-2]'] = np.log10(np.abs(df['XRT Column Density (NH) [10^21 cm^-2]'] + epsilon)) #bring back this one later if necessary

    return df

In [69]:
df_imputed_median_after = engineer_grb_features(df_imputed_median)
df_imputed_median_after.to_excel('median_imputation.xlsx', index=False)

In [70]:
df_imputed_knn_after = engineer_grb_features(df_imputed_knn)
df_imputed_knn_after.to_excel('knn_imputation.xlsx', index=False)

In [71]:
df_imputed_mice_after = engineer_grb_features(df_imputed_mice)
df_imputed_mice_after.to_excel('mice_imputation.xlsx', index=False)